In [132]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import seaborn as sns

In [133]:
account = pd.read_csv('data/account.csv', delimiter=";")
card_test = pd.read_csv('data/card_test.csv', delimiter=";")
card_train = pd.read_csv('data/card_train.csv', delimiter=";")
client = pd.read_csv('data/client.csv', delimiter=";")
disp = pd.read_csv('data/disp.csv', delimiter=";")
district = pd.read_csv('data/district.csv', delimiter=";")
loan_test = pd.read_csv('data/loan_test.csv', delimiter=";")
loan_train = pd.read_csv('data/loan_train.csv', delimiter=";")
trans_test = pd.read_csv('data/trans_test.csv', delimiter=";")
trans_train = pd.read_csv('data/trans_train.csv', delimiter=";")


/home/ricardo/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Account
- accound_id
- district_id
- date
- frequency

# Client
- client_id
- birth_number
- district_id

# Disposition
- disp_id
- client_id
- account_id
- type

# Loan
- loan_id
- account_id
- date
- amount
- duration
- payments
- status

# Transition
- trans_id
- account_id
- date
- type
- operation
- amount_balance
- k_symbol
- bank
- account

In [134]:
def getGender(df):
    List = []
    for row in df.itertuples(index = True):
        month = getMonth(row.birth_number)
        birth_number = row.birth_number
        gender = 'M'
        
        if(month > 50):
            birth_number -= 5000
            gender = 'F'
            
        List.append([row.client_id, birth_number, gender, row.district_id])
    
    return pd.DataFrame(List, columns=['client_id', 'birth_number', 'gender', 'district_id'])

In [135]:
def getMonth(date):
    removeDay = date //100
    month = removeDay % 100
    return month

In [136]:
from sklearn import tree
X_train = loan_train.drop(columns='status')
y_train = loan_train['status'].copy()
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)

X_test = loan_test.drop(columns='status')
y_test = clf.predict(X_test)

In [137]:
predict_loan = {'Id': X_test["loan_id"], 'Predicted': y_test}
df = pd.DataFrame(data=predict_loan)
df.to_csv('loan_predict.csv', index=False)

In [140]:
client_processed = getGender(client)
client_processed

,client_id,birth_number,gender,district_id
0,1,701213,F,18
1,2,450204,M,1
2,3,401009,F,1
3,4,561201,M,5
4,5,600703,F,5
...,...,...,...,...
5364,13955,451030,F,1
5365,13956,430406,M,1
5366,13968,680413,M,61
5367,13971,621019,F,67


In [141]:
client_processed

,client_id,birth_number,gender,district_id
0,1,701213,F,18
1,2,450204,M,1
2,3,401009,F,1
3,4,561201,M,5
4,5,600703,F,5
...,...,...,...,...
5364,13955,451030,F,1
5365,13956,430406,M,1
5366,13968,680413,M,61
5367,13971,621019,F,67
